In [1]:
import pyabf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [2]:
# these are a set of convenience functions for working with Homeira and Lihua's files 
from scripts.util_functions import guess_response_gain, get_stim_gain, get_stim_info, get_stim_dict

In [3]:
# read in cell metadata and processed ephys property dictionary
# this is needed because we need to know what the RMP values are per cell

summary_table_csv = '/Users/stripathy/rstudio_projects/valiante_ih/summary_tables/cell_metadata.csv'
#excel_file = file_base_base_path + 'valiante_lab_ephys_mar_2020/L23/Homeira/Total2Homeira-Lastversion_March 10_2020.xlsx'
cell_info = pd.read_csv(open(summary_table_csv, 'rb'))  
len(cell_info)

204

In [4]:
# define directories containing abf files from Homeira and Lihua's experiments

### change the directory paths below to where the files are on your local directory
file_base_base_path = '/Users/stripathy/Downloads/'
file_base_path = 'valiante_lab_ephys_mar_2020/'

path = file_base_base_path + file_base_path

# get all abf files in directories
files = [f for f in glob.glob(path + "**/*.abf", recursive=True)]


In [5]:
import os
print(os.path.basename(files[0]))

# traverse directories and start keeping track of important metadata
files_list = list()
for f in files:
    base_file_name = os.path.basename(f)
    ret_dict = {'cell_id' : base_file_name, 'full_path' : f}
    files_list.append(ret_dict)
    
file_dict = pd.DataFrame(files_list)
len(file_dict)

# drop file duplicates
file_dict = file_dict.drop_duplicates(subset=['cell_id'])
len(file_dict)
#file_dict.sort_values(by = ['abf_version'])

19122022.abf


230

In [6]:
# load the sheets containing gains and offsets provided by homeira
gain_base_dir = '/Users/stripathy/rstudio_projects/valiante_ih/raw-data/final_kri_data/gain_offset_info/'
gain_files = [f for f in glob.glob(gain_base_dir + "*.xlsx", recursive=False)]

pd_list = list()
for f in gain_files:
    gain_info = pd.read_excel(open(f, 'rb'))  
    pd_list.append(gain_info)
gain_info_df = pd.concat(pd_list, axis = 0)
gain_info_df.rename(columns={"File name": "cell_id", "Gain": "clampfit_gain", "Offset voltage" : "resp_offset"}, inplace=True)

/Users/stripathy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [7]:
# merge the metadata and file info pandas data frames
cell_meta_df = cell_info.merge(file_dict, on='cell_id', how='left')
cell_meta_df = cell_meta_df.merge(gain_info_df, on='cell_id', how='left')

In [8]:
cell_meta_df

,Unnamed: 0,cell_id,expt_date,layer_name,cell_type,recorder_name,subject_id,rmp,full_path,Command channel,Gain,resp_offset,Response channel,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,1,13n05011.abf,2013-11-05,L5,Pyr,Homeira,X2013.11.05,-54.937742,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
1,2,13n21007.abf,2013-11-21,L5,Pyr,Homeira,X2013.11.21,-59.158322,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,20X,0.0,200B_IC,NaN,NaN,NaN,NaN
2,3,13d02004.abf,2013-12-02,L5,Pyr,Homeira,X2013.12.02,-59.069821,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
3,4,13d02022.abf,2013-12-02,L5,Pyr,Homeira,X2013.12.02,-64.428708,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
4,5,13d02049.abf,2013-12-02,L5,Pyr,Homeira,X2013.12.02,-57.000000,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
5,6,13d03005.abf,2013-12-03,L5,Pyr,Homeira,X2013.12.03,-63.714597,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
6,7,13d03007.abf,2013-12-03,L5,Pyr,Homeira,X2013.12.03,-56.463620,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
7,8,13d03008.abf,2013-12-03,L5,Pyr,Homeira,X2013.12.03,-59.320066,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
8,9,13d03029.abf,2013-12-03,L5,Pyr,Homeira,X2013.12.03,-56.463620,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN
9,10,13d03040.abf,2013-12-03,L5,Pyr,Homeira,X2013.12.03,-60.000000,/Users/stripathy/Downloads/valiante_lab_ephys_...,No command signal,50X,0.0,200B_IC,NaN,NaN,NaN,NaN


In [9]:
# test the usage of the convience functions defined above on a single abf file

abf_file_name = '19129043.abf'
curr_row = cell_meta_df.loc[cell_meta_df['cell_id'] == abf_file_name]
get_stim_dict(curr_row, cell_meta_df)



IndexError: list index out of range

In [10]:
# test guess response gain function
abf_file_name = '15622019.abf'

curr_row = cell_meta_df.loc[cell_meta_df['cell_id'] == abf_file_name]
fn = curr_row['full_path'].values[0]
print(curr_row.keys())
rmp = curr_row['rmp'].values[0]
offset = curr_row['resp_offset'].values[0]
print(offset)
print(rmp)
abf = pyabf.ABF(fn) # loads in the abf file
print(abf)
abf.setSweep(0, channel=0)
resp_vec = abf.sweepY
print(resp_vec)
guess_response_gain(resp_vec, rmp, offset)

Index(['Unnamed: 0', 'cell_id', 'expt_date', 'layer_name', 'cell_type',
       'recorder_name', 'subject_id', 'rmp', 'full_path', 'Command channel',
       'Gain ', 'resp_offset', 'Response channel', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')
-104.1
-64.1
ABF (version 2.6.0.0) with 2 channels (mV, MV), sampled at 20.0 kHz, containing 50 sweeps, having 1 tag (C#6, RMP -64.1 mV, Normal aCSF), with a total length of 2.19 minutes, recorded with protocol "firings".
[42.0654 42.0837 42.0471 ... 41.8274 41.8457 41.8274]


(1.0, 2.0654258728027344)

In [19]:
# sweep through all listed abf files and extract all relevant metadata required to parse raw data

path = file_base_base_path + file_base_path

files = [f for f in glob.glob(path + "**/*.abf", recursive=True)]

#lihua_stim_abf = 
dict_list = list()

for abf_file in cell_meta_df.cell_id:
    #print(abf_file)
    row = cell_meta_df.loc[cell_meta_df['cell_id'] == abf_file]
    
    f = row['cell_id'].values[0]
    #
    fn = row['full_path'].values[0]
    #print(fn)
    resp_chan = 0
    stim_chan = 0
    stim_info_dict = {}
    
    cell_rmp = row['rmp'].values[0]
    resp_offset = row['resp_offset'].values[0]
    recorder_name = row['recorder_name'].values[0]

    
    abf = pyabf.ABF(fn) # loads in the abf file
    num_sweeps = abf.sweepCount
    abf_tag = abf.tagComments
    if abf_tag:
        abf_tag = abf_tag[0]
    else:
        abf_tag = None
    #print(recorder_name)
    #print(num_sweeps)
    
    if len(abf.channelList) > 2:
        # don't know what to do with these, skipping for now
#         print('skipping file because too many abf channels')
        
#         ret_dict = {'cell_id' : f, 'abf_version' : abf.abfVersionString, 
#                     'valid_stim': False
#                    }
#         dict_list.append(ret_dict)
        
#         continue
        stim_chan = 3
        abf.setSweep(0, channel=stim_chan)
        stim_vec = abf.sweepY
        
        resp_chan = 2
        abf.setSweep(0, channel=resp_chan)
        resp_vec = abf.sweepY
        # try to figure out gain on response channel by comparing to RMP
        (best_gain, rmp_abs_error) = guess_response_gain(resp_vec, cell_rmp, resp_offset)
        
        stim_gain = .2
        valid_stim = True
        stim_temp_dict = {'stim_chan' : stim_chan, 'stim_gain' : stim_gain, 'stim_path' : fn, 'valid_stim' : valid_stim}
        stim_dict = get_stim_info(abf, stim_channel_num = 3, stim_gain = stim_gain, stim_name = 'sweepY')
        stim_dict.update(stim_temp_dict)
        stim_name = 'sweepY'   
    else:

        # figure out response channel
        abf.setSweep(0, channel=resp_chan)
        resp_vec = abf.sweepY
        resp_sampling_rate = int(round(1/(abf.sweepX[2] - abf.sweepX[1]))) # manually calculate the sampling rate

        # try to figure out gain on response channel by comparing to RMP
        (best_gain, rmp_abs_error) = guess_response_gain(resp_vec, cell_rmp, resp_offset)

        stim_dict = get_stim_dict(row, cell_meta_df)
        stim_name = 'sweepC'
    
    # if in the odd case the number of sweeps doesn't match between resp vs stim file, then make them match
    if num_sweeps != stim_dict['num_sweeps']:
        print(abf_file)
        stim_dict['num_sweeps'] = num_sweeps
        stim_dict['stim_amp_vec'] = stim_dict['stim_amp_vec'][0:num_sweeps]
        
    ret_dict = {'cell_id' : f, 'resp_chan' : resp_chan, 
                    'resp_gain' : best_gain, 'rmp_error' : rmp_abs_error, 'abf_tag' : abf_tag,
                'rmp_val' : cell_rmp, 'resp_offset' : resp_offset,
                'abf_version' : abf.abfVersionString, 'resp_sampling_rate' : resp_sampling_rate, 'stim_name' : stim_name}
    #ret_dict = dict(ret_dict.items() + stim_info_dict.items())
    ret_dict.update(stim_dict)
    #print(ret_dict)

    dict_list.append(ret_dict)
    

cell_raw_file_meta_df = pd.DataFrame(dict_list)



14408304.abf
14515323.abf
14515424.abf


In [20]:
# merge together cell_raw_file_meta_df with some other cell metadata from the imported csv

cell_meta_df_small = cell_meta_df[['cell_id', 'expt_date', 'layer_name', 'cell_type', 'recorder_name', 'full_path']]
cell_final_raw_meta_df = cell_meta_df_small.merge(cell_raw_file_meta_df, on='cell_id', how='left')

In [21]:
cell_final_raw_meta_df = cell_final_raw_meta_df.sort_values(by = ['expt_date'])

In [22]:
# save a final csv that has the ouput of this metadata gathering process
csv_meta_save_path = 'output_files/cell_final_raw_meta_df.csv'
cell_final_raw_meta_df.to_csv(csv_meta_save_path)

In [15]:
df = cell_final_raw_meta_df.query('rmp_error > 10')

In [16]:
df[['cell_id', 'abf_tag', 'rmp_error', 'resp_gain']]

,cell_id,abf_tag,rmp_error,resp_gain
201,2020_01_06_0095.abf,"C13, L3, RMP -60.2,",12.401318,1.0
